In [1]:
import numpy as np
import pandas as pd
import math
import os.path
from os import walk
import ntpath

In [2]:
t="C:/Users/Zaineb/Documents/WORK/METEO/PFE/new_data"
listeFichiers = []
for fichiers in walk(t):
 listeFichiers.extend(fichiers)

print("liseFichiers=", listeFichiers)

liseFichiers= ['C:/Users/Zaineb/Documents/WORK/METEO/PFE/new_data', [], ['arome_2021010100.xlsx', 'arome_2021010200.xlsx']]


In [3]:
L=listeFichiers[2]
L[0][6:-7]

'20210101'

In [4]:
#Les fonctions de IFM:

#ICL prend la température, l'humidité relative, la vitesse du vent, la pluie et une valeur ICL précédente pour produire la valeur ICL actuelle
#    - ICL(17,42,25,0,85) = 87.692980092774448

#IH prend la température, l'humidité relative, les précipitations, la valeur IH précédente, la latitude et le mois en cours pour produire la valeur IH actuelle
#    - IH(17,42,0,6,45.98,4) = 8.5450511359999997

#IS prend la température, la pluie, la valeur IS précédente, la latitude et le mois en cours pour produire la valeur IS actuelle
#   - DC(17,0,15,45.98,4) = 19.013999999999999

#IPI prend la vitesse du vent et la valeur ICL actuelle pour produire la valeur IPI actuelle
#   - ISI(25,87.692980092774448) = 10.853661073655068

#ICD prend les valeurs IH et IS actuelles pour produire la valeur ICD actuelle
#   - ICD(8.5450511359999997,19.013999999999999) = 8.4904265358371838

#IFM prend les valeurs IPI et ICD actuelles pour produire la valeur IFM actuelle
#   - IFM(10.853661073655068,8.4904265358371838) = 10.096371392382368

#calcIFM - cette fonction renvoie la valeur IFM actuelle en fonction de toutes les valeurs d'entrée : mois, température, humidité relative, vitesse du vent, pluie, ICL précédent, IH, IS et latitude
#   - calcIFM(4,17,42,25,0,85,6,15,45.98) = 10.096371392382368

In [5]:
class InvalidLatitude(Exception):
    """Exception pour gérer les variables non couvertes par DataDict"""
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return repr(self.value) + " n'est pas une latitude valide."

In [6]:
def ICL(T,H,FF,RR,ICLPrev):
    '''Calcule le code d'humidité du carburant fin d'aujourd'hui
    PARAMETRES
    ----------
    * T est la température de 12:00 LST en degrés Celsius
    * H est l'humidité relative de 12:00 LST en %
    * FF est la vitesse du vent à 12:00 LST en km/h
    * RR est la pluie cumulée sur 24 heures en mm, calculée à 12:00 LST
    * ICLPrev est l'ICL de la veille'''

    H = min(100.0,H)
    mo = 147.2 * (101.0 - ICLPrev) / (59.9 + ICLPrev)

    if RR > 0.5:
        rf = RR - 0.5

        if mo <= 150.0: 
            mr = mo + \
                 42.5 * rf * math.exp(-100.0 / (250.0 - mo)) * (1.0-math.exp(-6.93 / rf))
        else:

            mr = mo + \
                 42.5 * rf * math.exp(-100.0 / (251.0 - mo)) * (1.0-math.exp(-6.93 / rf)) + \
                 0.0015 * pow(mo - 150.0, 2) * pow(rf, 0.5)

        if mr > 250.0:
            mr = 250.0

        mo=mr

    ed = 0.942 * pow(H, 0.679) + \
         11.0 * math.exp((H - 100.0) / 10.0) + 0.18 * (21.1 - T) * (1.0 - math.exp(-0.115 * H))

    if mo > ed:
        ko = 0.424 * (1.0 - pow(H / 100.0, 1.7)) + \
             0.0694 * pow(FF, 0.5) * (1.0 - pow(H / 100.0, 8))

        kd = ko * 0.581 * math.exp(0.0365 * T)

        m = ed + (mo - ed) * pow(10.0,-kd)

    else:
        ew = 0.618 * pow(H,0.753) + \
             10.0 * math.exp((H - 100.0) / 10.0) + \
             0.18 * (21.1 - T) * (1.0 - math.exp(-0.115 * H))
        if mo < ew:
            k1 = 0.424 * (1.0 - pow((100.0 - H) / 100.0, 1.7)) + \
                 0.0694 * pow(FF, .5) * (1.0 - pow((100.0 - H) / 100.0, 8))

            kw = k1 * 0.581 * math.exp(0.0365 * T)


            m = ew - (ew - mo) * pow(10.0, -kw)
        else:
            m = mo
    return 59.9 * (250.0 - m) / (147.2 + m)

In [7]:
def DayLength(Latitude, MONTH):
    '''Estimation de la durée du jour en fonction du mois et de la latitude'''

    DayLength46N = [ 6.5,  7.5,  9.0, 12.8, 13.9, 13.9, 12.4, 10.9,  9.4,  8.0,  7.0,  6.0]
    DayLength20N = [ 7.9,  8.4,  8.9,  9.5,  9.9, 10.2, 10.1,  9.7,  9.1,  8.6,  8.1,  7.8]
    DayLength20S = [10.1,  9.6,  9.1,  8.5,  8.1,  7.8,  7.9,  8.3,  8.9,  9.4,  9.9, 10.2]
    DayLength40S = [11.5, 10.5,  9.2,  7.9,  6.8,  6.2,  6.5,  7.4,  8.7, 10.0, 11.2, 11.8]

    retVal = None

    if Latitude<= 90 and Latitude > 33:
        retVal = DayLength46N[MONTH-1]
    elif Latitude <= 33 and Latitude > 0.0:
        retVal = DayLength20N[MONTH-1]
    elif Latitude <= 0.0 and Latitude > -30.0:
        retVal = DayLength20S[MONTH-1]
    elif Latitude <= -30.0 and Latitude >= -90.0:
        retVal = DayLength40S[MONTH-1]

    if retVal==None:
        raise InvalidLatitude(Latitude)

    return retVal

In [8]:
def IH(T,H,RR,IHPrev,LAT,MONTH):
    ''' Calcule le code d'humidité Duff d'aujourd'hui
    PARAMÈTRES
    ----------
    * T est la température de 12:00 LST en degrés Celsius
    * H est l'humidité relative de 12:00 LST en %
    * RR est la pluviométrie cumulée sur 24 heures en mm, calculée à 12h00 LST
    * IHPrev est l'IH de la veille
    * Lat est la latitude en degrés décimaux de l'emplacement pour lequel les calculs sont effectués
    * Month est le mois de l'année (1..12) pour les calculs du jour en cours.'''

    H = min(100.0,H)
    if RR > 1.5:
      re = 0.92 * RR - 1.27
      mo = 20.0 + 280* math.exp(-0.023 * IHPrev)
      if IHPrev <= 33.0:
        b = 100.0 / (0.5 + 0.3 * IHPrev)
      else:
        if IHPrev <= 65.0:
          b = 14.0 - 1.3 * math.log(IHPrev)
        else:
          b = 6.2 * math.log(IHPrev) - 17.2
        
        mr = mo + 1000.0 * re / (48.77 + b * re)

        pr = 244.72 - 43.43 * math.log(mr - 20.0)

        if pr > 0.0:
            IHPrev = pr
        else:
            IHPrev = 0.0

    if T > -1.1:
      """ DayLength calcule la longueur du jour (le temps entre le lever du soleil et coucher du soleil) compte tenu du jour de l'année et de la latitude du lieu."""
      d1 = DayLength(LAT,MONTH)
      k = 1.894 * (T + 1.1) * (100.0 - H) * d1 * 0.000001

    else:
      k = 0.0

    return IHPrev + 100.0 * k

In [9]:
def DryingFactor(Latitude, Month):

    LfN = [-1.6, -1.6, -1.6, 0.9, 3.8, 5.8, 6.4, 5.0, 2.4, 0.4, -1.6, -1.6]
    LfS = [6.4, 5.0, 2.4, 0.4, -1.6, -1.6, -1.6, -1.6, -1.6, 0.9, 3.8, 5.8]

    if Latitude > 0:
        retVal = LfN[Month]
    elif Latitude <= 0.0:
        retVal = LfS[Month]

    return retVal

In [10]:
def IS(T,RR,ISPrev,LAT,MONTH):
    '''Calcule les paramètres du code de sécheresse d'aujourd'hui
    PARAMÈTRES
    ----------
    T est la température de 12:00 LST en degrés Celsius
    RR est la pluviométrie cumulée sur 24 heures en mm, calculée à 12h00 LST
    ISPrev est le DC de la veille
    LAT est la latitude en degrés décimaux de l'emplacement pour lequel les calculs sont effectués
    MONTH est le mois de l'année (1..12) pour les calculs du jour en cours'''

    if RR > 2.8:
        rd = 0.83 * RR - 1.27
        Qo = 800.0 * math.exp(-ISPrev / 400.0)
        Qr = Qo + 3.94 * rd
        Dr = 400.0 * math.log(800.0 / Qr)
        
        if Dr > 0.0:
            ISPrev = Dr
        else:
            ISPrev = 0.0

    Lf = DryingFactor(LAT, MONTH-1)

    if T > -2.8:
        V = 0.36 * (T+2.8) + Lf
    else:
        V = Lf
    
    if V < 0.0:
        V = 0.0

    D = ISPrev + 0.5 * V

    return D

In [11]:
def IPI(FF,ICL):
    '''Calcule l'indice de propagation initial d'aujourd'hui
    PARAMÈTRES
    ----------
    FF est la vitesse du vent à 12:00 LST en km/h
    ICL est le FFMC du jour'''

    fF0 = math.exp(0.05039 * FF)

    m = 147.2 * (101.0 - ICL) / (59.5 + ICL)

    fF1 = 91.9 * math.exp(-0.1386 * m) * (1.0 + pow(m, 5.31) / 49300000.0)

    return 0.208 * fF0 * fF1 

In [12]:
def ICD(IH,IS):
    '''Calcule l'indice d'accumulation d'aujourd'hui
    PARAMÈTRES
    ----------
    IH est le code d'humidité Duff du jour en cours
    IS est le code de sécheresse du jour en cours'''

    if IH <= 0.4 * IS:
        U = 0.8 * IH * IS / (IH + 0.4 * IS)
    else:
        U = IH - (1.0 - 0.8 * IS / (IH + 0.4 * IS)) * \
            (0.92 + pow(0.0114 * IH, 1.7))

    return max(U,0.0)

In [13]:
def IFM(IPI, ICD):
    '''Calcule l'indice météo d'incendie d'aujourd'hui
    PARAMÈTRES
    ----------
    IPI est l'IPI du jour en cours
    ICD est l'ICD du jour en cours'''

    if ICD <= 80.0:
        I = 0.626 * pow(ICD, 0.809) + 2.0
    else:
        I = 1000.0 / (25.0 + 108.64 * math.exp(-0.023 * ICD))

    B = 0.1 * IPI * I

    if B >= 1.0:
        S = 2.72 * (pow(0.434 * math.log(B), 0.647))
    else:
        S = B

    return S

In [14]:
def calcIFM(MONTH,T,H,FF,RR,ICLPrev,IHPrev,ISPrev,LAT):
    '''Calcule le IFM d'aujourd'hui
    PARAMÈTRES
    ----------
    MONTH est le mois numérique, de 1 à 12
    T est la température de 12:00 LST en degrés Celsius
    H est l'humidité relative de 12:00 LST en %
    FF est la vitesse du vent à 12:00 LST en km/h
    RAIN est la pluviométrie cumulée sur 24 heures en mm, calculée à 12h00 LST
    ICLPrev est le ICL de la veille
    IHPrev est le IH de la veille
    ISPrev est le IS de la veille
    LAT est la latitude en degrés décimaux de l'emplacement pour lequel les calculs sont effectués'''

    icl = ICL(T,H,FF,RR,ICLPrev)
    ih  = IH(T,H,RR,IHPrev,LAT,MONTH)
    iss = IS(T,RR,ISPrev,LAT,MONTH)
    ipi = IPI(FF, icl)
    icd = ICD(ih,iss)
    ifm = IFM(ipi, icd)

    return ifm

In [15]:
for k in range(len(L)):
    try:
        df=pd.read_excel("C:/Users/Zaineb/Documents/WORK/METEO/PFE/new_data/" + L[k])
        tab=[]
        for i in range(len(df)):
            stat= df['STATION'][i]
            p1= ICL(df['MAXI(J)'][i], df['HRmax'][i],df['FFmax'][i], df['RRtot'][i], 0 )
            p2= IH(df['MAXI(J)'][i], df['HRmax'][i], df['RRtot'][i], 0 , 0 , 0 )
            p3= IS(df['MAXI(J)'][i], df['RRtot'][i], 0 , 0 , 0 )
            p4= IPI(df['FFmax'][i], p1 )
            p5= ICD(p2, 0 )
            p6= IFM(p4,p5)
            if p6 >=0 and p6 < 10:
                p7= "DANGER FAIBLE"
            if p6>=10 and p6 < 30:
                p7= "DANGER LEGER"
            if p6>=30 and p6 < 50:
                p7= "DANGER MODERE"
            if p6>=50 and p6 < 80:
                p7= "DANGER SEVERE"
            if p6>=80: 
                p7= "DANGER TRES SEVERE"
                
            tab.append([stat,p1,p2,p3,p4,p5,p6,p7])
            
        df1 = pd.DataFrame (tab, columns = ['STATION','ICL','IH','IS','IPI','ICD','IFM','CONCLUSION'])
        
        df1.to_csv('Indices_'+L[k][6:-7]+'00.csv')
        
        if L[k][10:-9] in {'01','03','05','07','08','10','12'}:
            if int(L[k][12:-7]) < 31:
                x=int(L[k][12:-7])
                x=+1
                L[k]=L[k][0:12]+str(x)+"00.xlsx"
            else:
                y=int(L[k][10:-8])
                y=+1
                L[k]=L[k][0:12]+str(y)+"00.xlsx"
        elif L[k][10:-9] in {'04','06','09','11'}:
            if int(L[k][12:-7]) < 30:
                x=int(L[k][12:-7])
                x=+1
                L[k]=L[k][0:12]+str(x)+"00.xlsx"
            else:
                y=int(L[k][10:-9])
                y=+1
                L[k]=L[k][0:12]+str(y)+"00.xlsx"
        else:
            if int(L[k][12:-7]) < 28:
                y=int(L[k][10:-9])
                y=+1
                L[k]=L[k][0:12]+str(y)+"00.xlsx"
        
    except IOError:
        print("Erreur ! Le fichier n'existe pas encore")